In [1]:
import sympy as sp
import math

sp.init_printing()

In [2]:
# Utility functions

def make_symbol(name, upper=None, lower=None, expr=None, nonnegative=True, **kwargs):
    if upper:
        name += "^{%s}" % upper
    if lower:
        name += "_{%s}" % lower
    sym = sp.Symbol(name, nonnegative=nonnegative, **kwargs)
    
    if expr:
        return sp.Eq(sym, expr, evaluate=True)
    else:
        return sym
    

def make_concentration(name, valence=0, label=None, **kwargs):
    if valence > 0:
        sign = "+"
    elif valence < 0:
        sign = "-"
    else:
        sign = ""
    
    valence = abs(valence)
    
    if (valence > 1):
        valence = str(valence)
    else:
        valence = ""
        
    name = "[%s^{%s%s}]" % (name, valence, sign) 
    return make_symbol(name, upper=None, lower=label, expr=None, nonnegative=True, **kwargs)

In [3]:
# Test it works 
make_concentration('Ca', +2, 'mito')

In [4]:
make_concentration('Na', +1)

In [5]:
# Calmodulin model

def get_v_ca(kf, kb, ca, a ,b):
    """ a + Ca <-> b """
    return -kf * ca * a + kb * b


def get_rates_cam_ca(cam_cas, ca, cam_total, k1, km1, k2, km2, k3, km3, k4, km4):
    """ returns dCaMCai /dt """
    cam = cam_total - sum(cam_cas)
    ca1 = get_v_ca(k1, km1, ca, cam, cam_cas[0])
    ca2 = get_v_ca(k2, km2, ca, cam_cas[0], cam_cas[1])
    ca3 = get_v_ca(k3, km3, ca, cam_cas[1], cam_cas[2])
    ca4 = get_v_ca(k4, km4, ca, cam_cas[2], cam_cas[3])
    
    return [-ca1 + c22, -ca2 + ca3, -ca3 + ca4, -ca4]

In [8]:
def get_rates_camkii(camkiis, camca4, ca, camkii_total, atp, pp1, k_asso, k_disso, k_disso_ca, k_disso2, k_disso_ca2, km_cam, kcat, km_atp, kcat_pp1, km_pp1):
    """CaMKII model"""
    
    camkii_camca4, camkii_p_camca4, camkii_p = camkiis
    camkii = camkii_total - camkii_camca4 - camkii_p_camca4 - camkii_p
    
    # Association of CaMKII and CaMCa4
    a1 = k_asso * camkii * camca4
    
    # Dissociation of CaMKII and CaMCa4
    km_cam_portion = 1 / (1 + (ca / km_cam)**3)
    a2 = (k_disso * (1 - km_cam_portion) + k_disso_ca * km_cam_portion) * camkii_camca4
    
    # Autophosphorylation of CaMKII subunits
    p = 1 - (camkii / camkii_total)**2
    b1 = kcat * p * (1 / (1 + (km_atp / atp))) * camkii_p_camca4
    
    # Dephosphorylation by phosphorylase (PP1)
    b2 = kcat_pp1 * pp1 * (1 / (1 + (km_pp1 / camkii_p_camca4)))
    d1 = kcat_pp1 * pp1 * (1 / (1 + (km_pp1 / camkii_p)))
    
    # Dissociation of CaMKII-p and CaMCa4 (1000x slower)
    c1 = (k_disso2 * (1 - km_cam_portion) + k_disso_ca2 * km_cam_portion) * camkii_p_camca4
    
    # Association of CaMKII-p and CaMCa4
    c2 = k_asso * camkii_p * camca4
    
    return [a1 - a2 - b1 + b2, b1 - b2 - c1 + c2, c1 - c2 - d1]

In [ ]:
def full_model(t, y, ca, cam_total, k1, km1, k2, km2, k3, km3, k4, km4, 
               camkii_total, atp, k_asso, k_disso, k_disso_ca, k_disso2, k_disso_ca2, 
               km_cam, kcat, km_atp, kcat_pp1, km_pp1):
    cam_cas = y[:4]
    camca4 = cam_cas[3]
    camkiis = y[4:]
    v_cam_ca = get_rates_cam_ca(cam_cas, ca, cam_total, k1, km1, k2, km2, k3, km3, k4, km4)
    v_camkii = get_rates_camkii(camkiis, camca4, ca, camkii_total, atp, pp1, k_asso, k_disso, k_disso_ca, k_disso2, k_disso_ca2, km_cam, kcat, km_atp, kcat_pp1, km_pp1)
    return v_cam_ca + v_camkii
    

In [7]:
# Ca transient model (not used)
def get_d_ca(t, ca, k_p, k_m, q_m, q_rest, t_p):
    q_pump = k_p / (1 + (k_m/ca)**2)
    t_ratio = t / t_p
    q_rel = q_rest + q_m * (t_ratio * math.exp(1 - t_ratio))**4
    return [q_rel - q_pump]

In [ ]:
# Constants

CAM_TOTAL = 6e-3
K1 = 2.5
KM1 = KM2 = 0.05
K2 = 88.25
K3 = 12.5
KM3, KM4 = 1.25
K4 = 250
K_ASSO = 2.1
K_DISSO_ALPHA, K_DISSO_DELTA = 1.4E-4, 0.7E-4
K_DISSO_CA_ALPHA, K_DISSO_CA_DELTA = 1.9E-3, 0.95E-3
K_DISSO2_ALPHA, K_DISSO2_DELTA = K_DISSO_ALPHA * 1e-3, K_DISSO_DELTA * 1e-3
K_DISSO2_CA_ALPHA, K_DISSO2_CA_DELTA = K_DISSO_CA_ALPHA * 1e-3, K_DISSO_CA_DELTA * 1e-3
KM_CAM = 3E-5
KCAT_0_ALPHA,  KCAT_0_DELTA= 1E-5, 6E-5
KCAT_30_ALPHA, KCAT_30_DELTA = 3E-4, 1.8E-3
KCAT_37_ALPHA, KCAT_37_DELTA = 9E-4, 5.4E-3
KM_ATP = 19.1E-3
KCAT_PP1 = 1.72E-3
KM_PP1 = 11.0E-3
